# Toy Data Solution

In [ ]:
import numpy as np
#import pandas as pd
import torch
import torch.nn
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve

def reseed(seed=96):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

# 1.1 Generating synthetic data

Make function `generate_toydata()` that recieves:
* `batch_size` - number of datapoints
* `w` - multipliyer of linear function
* `b` - offset of linear function

and returns two vectors:
* `X` of shape (batch_size, 2) containing 2D datapoints
* `Y` if shape (batch_size,) containing targets.

This function generates random batch size number of 2D points $(x_1, x_2)$ where:
* $x_1$ are random numbers and
* $x_2$ are random numbers multiplied by $w$ and to which $b$ is added.

Target ($y$) of $(x_1, x_2)$ will be:
* $1$ if $x_2 > w * x_1 + b$
* else $0$



In [ ]:
def generate_toy_data(batch_size, w, b):
    return

In [ ]:
reseed(1337)
temp = generate_toy_data(4, w=2, b=1)
assert isinstance(temp, tuple) and len(temp) == 2
assert torch.allclose(temp[0], torch.tensor([[0.0783, 1.4008], [0.4956, 1.0573], [0.6231, 2.1702], [0.4224, 2.3934]]), atol=1e-4)
assert torch.equal(temp[1], torch.tensor([1., 0., 0., 1.]))
del temp


# 1.2 Plotting data

Create function `plot_toy_data()` which receives two vectors:
* $X$ vector of n datapoints $(x_1, x_2)$
* $Y$ vector of n targets

This function plots 2D datapoints in blue if corresponding target value is 1, red if opposite.

Generate 1000 synthetic data points where $w$ = 5 and $b$ = 3.

In [ ]:
def plot_toy_data(x_data, y_truth, perceptron=None):
    blue = []
    orange = []
    black_blue = []
    black_orange = []
    if perceptron:
        y_pred = perceptron(x_data).squeeze().detach()
        y_pred = (y_pred > 0.5).float()
    else:
        y_pred = y_truth

    for x_i, y_true_i, y_pred_i in zip(x_data, y_truth, y_pred):
        
        is_black = y_true_i != y_pred_i

        if y_true_i == 1.:
            if is_black:
                black_blue.append(x_i)
            else:
                blue.append(x_i)
        else:
            if is_black:
                black_orange.append(x_i)
            else:
                orange.append(x_i)
    
    if blue:
        blue = np.stack(blue)
        plt.scatter(blue[:,0], blue[:,1], marker=".", c="tab:blue")
    
    if orange:
        orange = np.stack(orange)
        plt.scatter(orange[:,0], orange[:,1], marker=".", c="tab:red")

    if perceptron:
        if black_blue:
            black_blue = np.stack(black_blue)
            plt.scatter(black_blue[:,0], black_blue[:,1], marker=".", c="black")
        if black_orange:
            black_orange = np.stack(black_orange)
            plt.scatter(black_orange[:,0], black_orange[:,1], marker=".", c="black")

        # hyperplane
        xx = np.linspace(x_data[:,0].min(), x_data[:,0].max(), 30)
        yy = np.linspace(x_data[:,1].min(), x_data[:,1].max(), 30)
        xv, yv = np.meshgrid(xx, yy)
        xy = np.vstack([xv.ravel(), yv.ravel()]).T
        z = perceptron(torch.tensor(xy, dtype=torch.float)).detach().numpy().reshape(yv.shape)
        
        plt.contour(xx, yy, z, colors='k', linestyles=["--", "-", "--"], levels=[0.4, 0.5, 0.6])
    plt.show()

plot_toy_data(*generate_toy_data(1024, w=5, b=3))
# plot_toy_data(*get_toy_data(1024), perceptron)


# 2.1. Perceptron

Create `Perceptron` class where perceptron receives 2 numbers and outputs 1 number.  
Create `forward()` method which receives datapoint $(x_1, x_2)$ which is an input to perceptron and applies sigmoid on perceptron's output.  
Create `predict()`method which receives datapoint $(x_1, x_2)$ and returns 1 if the result after forward is greater or equal 0.5, otherwise 0.  
Create `reset()` method which resets parameters of the model (call `reset_parameter` on model layer)

In [ ]:
class Perceptron(torch.nn.Module):

    def __init__(self):
        pass

    def forward(self, x_in):
        return 
    
    def predict(self, x_in):
        return 
    
    def reset(self):
        pass

In [ ]:
reseed(1337)
temp = Perceptron()
assert hasattr(temp, "forward") and hasattr(temp, "predict") and hasattr(temp, "reset")
assert torch.allclose(temp(torch.FloatTensor([1, 2])), torch.tensor(0.3930), atol=1e-4)
assert torch.allclose(temp(torch.FloatTensor([[1, 2], [3, 4]])), torch.tensor([0.3930, 0.1625]), atol=1e-4)
assert torch.equal(temp.predict(torch.FloatTensor([1, 2])), torch.tensor(0.))
assert torch.equal(temp.predict(torch.FloatTensor([[1, 2], [-5, 4]])), torch.tensor([0., 1.]))

temp.reset()
W, b = temp.parameters()
assert torch.allclose(W, torch.tensor([-0.1097, -0.4237]), atol=1e-4) and torch.allclose(b, torch.tensor([-0.6666]), atol=1e-4)

del temp, W, b

Set following variables:
* learning rate `lr` to 0.01
* `batch_size` = 1000
* `W` = 90
* `B` = 6

Instantiate:
* `model` as perceptron, 
* `optimizer`as adam optimizer with defined learning rate, 
* `loss_fn` as binary cross-entropy loss, 

Generate and remember static toy data with predefined parameters into `x_data_static` and `y_truth_static`.
Plot toy data

In [ ]:
assert isinstance(model, Perceptron)
assert isinstance(optimizer, torch.optim.Adam)
assert optimizer.defaults["lr"] == 0.01

assert isinstance(loss_fn, torch.nn.BCELoss)

assert isinstance(x_data_static, torch.Tensor)
assert x_data_static.shape == (1000, 2)

assert isinstance(x_data_static, torch.Tensor)
assert y_truth_static.shape == (1000,)


# 2.2 Train

Set following variables:
* `n_epochs` to 100  - number of epochs
* `n_batches` to 10 - number of batches
* `epoch` to 0 - current epoch number

**Train procedure**

for each epoch  
then for each batch:
* make forward step over newly generate batch with parameters `batch_size`, `W` and `B`, 
* make backward step,
* append to `losses` current loss  

at the end of batch loop increment `epoch`.

Plot losses

# 2.3 Evaluate

Evaluate model on static data. calculate accuracy, precission, recall and f1. Printout confusion matrix.

Note: You can use `scikitlearn.metrics`

# 2.4. Interspect

Determine learnt weights and compare them to `W` and `B` parameters used for generating toy data.